# Parse SEC 10-Q

In [1]:
import pandas as pd
import numpy as np
import time, sys, os, re
import datetime, pickle
import getpass
from tqdm import tqdm
import datetime
from datetime import datetime

In [3]:
from __future__ import division, unicode_literals 
import codecs

# Import requests to retrive Web Urls example HTML. TXT 
import requests

# Import BeautifulSoup
from bs4 import BeautifulSoup

# import re module for REGEXes
import re

In [490]:
with open("/home/dada/MA_Minus/10-Q/LCNB/LCNB-0") as fp:
    document= BeautifulSoup(fp.read()).get_text().lower()

In [561]:
# Write the regexdocument= BeautifulSoup(fp.read()).get_text()
regex = re.compile('(item 2)|(item 3)') 

# Use finditer to math the regex
matches = regex.finditer(document)

In [542]:
# Create the dataframe
test_df = pd.DataFrame([(x.start(), x.end()) for x in matches])

In [554]:
def cleanStr(test_df_1):
    test_df_1 = re.sub('\xa0', '', test_df_1)
    test_df_1 = re.sub('\n','', test_df_1)
    test_df_1 = re.sub('\n\n','', test_df_1)
    test_df_1 = re.sub('\n\n\n','', test_df_1)
    test_df_1 = re.sub('\t', '', test_df_1)
    test_df_1 = re.sub(' ·', '', test_df_1)
        
    return test_df_1

In [432]:
#parse 10-Q MA-
ma_neg = os.listdir("/home/dada/MA_Minus/10-Q")
len(ma_neg)

112

In [ ]:
MA_Minus_df = pd.DataFrame()

for i in tqdm(sorted(ma_neg)):
    for j in sorted(os.listdir("/home/dada/MA_Minus/10-Q/{}".format(i))):
        
        with open("/home/dada/MA_Minus/10-Q/{}/{}".format(i, j)) as fp:
            document = BeautifulSoup(fp.read()).get_text().lower()

        if "item 2" in document:
            regex_i_j = regex.finditer(document)
            match_df = pd.DataFrame([(x.start(), x.end()) for x in regex_i_j])
            parse_i_j = BeautifulSoup(document[match_df.iloc[0,1]:match_df.iloc[len(match_df)-1,0]], 'lxml')
            parse_i_j = parse_i_j.get_text("\n\n")
            parse_i_j = cleanStr(parse_i_j)
            MA_Minus_i_j = pd.DataFrame([{"Ticker":i,"Period":j,"MDAA":parse_i_j}])
            
        else:
            continue
            
    MA_Minus_df = MA_Minus_df.append(MA_Minus_i_j)    

 46%|████▋     | 52/112 [18:13<17:55, 17.93s/it]

In [ ]:
print(MA_Minus_df.shape)
MA_Minus_df.head()

In [ ]:
#parse 10-Q MA+
ma_pos = os.listdir("/home/dada/MA_Plus/10-Q")
len(ma_pos)

In [ ]:
MA_Plus_df = pd.DataFrame()

for i in tqdm(sorted(ma_pos)):
    for j in sorted(os.listdir("/home/dada/MA_Plus/10-Q/{}".format(i))):
        
        with open("/home/dada/MA_Plus/10-Q/{}/{}".format(i, j)) as fp:
            document = BeautifulSoup(fp.read()).get_text().lower()

        if "item 2" in document:
            regex_i_j = regex.finditer(document)
            match_df = pd.DataFrame([(x.start(), x.end()) for x in regex_i_j])
            parse_i_j = BeautifulSoup(document[match_df.iloc[0,1]:match_df.iloc[len(match_df)-1,0]], 'lxml')
            parse_i_j = parse_i_j.get_text("\n\n")
            parse_i_j = cleanStr(parse_i_j)
            MA_Plus_i_j = pd.DataFrame([{"Ticker":i,"Period":j,"MDAA":parse_i_j}])
            
        else:
            continue
            
    MA_Plus_df = MA_Plus_df.append(MA_Plus_i_j)    

In [ ]:
MA_Plus_df["MA"] = 1
MA_Minus_df["MA"] = 0

In [ ]:
MA_all = MA_Minus_df.append(MA_Plus_df)

In [ ]:
pickle.dump(MA_all, open("MA_all.pkl", 'rb'))